# BẮT ĐẦU NHANH
Phần này sẽ giới thiệu toàn bộ các API phổ biến trong học máy (**machine learning**). Tham khảo các liên kết trong mỗi phần để tìm hiểu sâu hơn.
## Làm việc với dữ liệu (Data)
Pytorch có hai chương trình căn bản làm việc với data là `torch.utils.data.DataLoader` và `torch.utils.data.Dataset`. `Dataset` lưu các mẫu (**samples**) và nhãn (**labels**) tương ứng. `DataLoader` đóng gói một cách tuần tự xung quanh `Dataset`.

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

PyTorch cung cấp các thư viện dành riêng cho miền riêng biệt, chẳng hạn như [TorchText](https://pytorch.org/text/stable/index.html), [TorchVision](https://pytorch.org/vision/stable/index.html), và [TorchAudio](https://pytorch.org/audio/stable/index.html), tất cả đều bao gồm bộ dữ liệu. Đối với hướng dẫn này, chúng tôi sẽ sử dụng tập dữ liệu `TorchVision`.

Mô-đun `torchvision.datasets` chứa các đối tượng `Dataset` cho nhiều dữ liệu trong thế giới thực như **CIFAR**, **COCO** ([danh sách đầy đủ tại đây](https://pytorch.org/vision/stable/datasets.html)). Trong hướng dẫn này, chúng tôi sử dụng tập dữ liệu **FashionMNIST**. Mỗi Tập dữ liệu `TorchVision` đều bao gồm hai đối số: `transform` và `target_transform` để sửa đổi các mẫu và nhãn tương ứng.

In [2]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

Chúng ta truyền `Dataset` làm đối số cho `DataLoader`. Điều này bao bọc một tệp có thể lặp lại trên tập dữ liệu của chúng ta và hỗ trợ tự động phân lô, lấy mẫu, xáo trộn và tải dữ liệu đa luồn. Ở đây chúng tôi xác định kích thước lô `batch_size=64`, tức là mỗi phần tử trong `dataloader` có thể lặp lại sẽ trả về một lô gồm 64 tính năng và nhãn.

In [3]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break

Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64


## Tạo Models
Để định nghĩa mạng nơron trong PyTorch, chúng ta tạo một lớp kế thừa từ `nn.Module`. Chúng tôi xác định các layer của mạng trong hàm `__init__` và chỉ định cách dữ liệu sẽ truyền qua mạng trong hàm `forward`. Để tăng tốc các hoạt động trong mạng nơ-ron, chúng tôi chuyển nó sang GPU nếu có.

In [4]:
# Get cpu or gpu device for training
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


## Tối ưu thông số mô hình
Để đào tạo một mô hình, chúng ta cần một hàm mất mát ([loss function](https://pytorch.org/docs/stable/nn.html#loss-functions) và một bộ tối ưu hóa ([optimizer](https://pytorch.org/docs/stable/optim.html)).

In [5]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

Trong một vòng lặp đào tạo duy nhất, mô hình đưa ra dự đoán trên tập dữ liệu đào tạo (được cung cấp cho tập dữ liệu đào tạo theo lô) và Lan truyền ngược (**backpropagates**) dự đoán để điều chỉnh thông số của mô hình.

In [6]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch*len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:5d}]")        

Chúng tôi cũng kiểm tra hiệu suất của mô hình dựa trên tập dữ liệu thử để đảm bảo nó đang học hỏi.

In [7]:
def test(dataloader, model):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):0.1f}%, Avg loss: {test_loss:>8f} \n")

Quá trình đào tạo được thực hiện qua nhiều lần lặp lại (epoch). Trong mỗi epoch, mô hình học các thông số để đưa ra dự đoán tốt hơn. Chúng ta in độ chính xác (accuracy ) và mất mát (loss ) của mô hình ở mỗi epoch; chúng ta muốn thấy độ chính xác tăng lên và tổn thất giảm theo từng epoch.

In [8]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model)
print("Done!")

Epoch 1
------------------------
loss: 2.300446 [    0/60000]
loss: 2.287151 [ 6400/60000]
loss: 2.280784 [12800/60000]
loss: 2.279829 [19200/60000]
loss: 2.263792 [25600/60000]
loss: 2.272288 [32000/60000]
loss: 2.255639 [38400/60000]
loss: 2.252787 [44800/60000]
loss: 2.251037 [51200/60000]
loss: 2.233919 [57600/60000]
Test Error: 
 Accuracy: 40.9%, Avg loss: 0.035045 

Epoch 2
------------------------
loss: 2.232817 [    0/60000]
loss: 2.207073 [ 6400/60000]
loss: 2.189156 [12800/60000]
loss: 2.203077 [19200/60000]
loss: 2.170915 [25600/60000]
loss: 2.194264 [32000/60000]
loss: 2.164804 [38400/60000]
loss: 2.161633 [44800/60000]
loss: 2.171298 [51200/60000]
loss: 2.128254 [57600/60000]
Test Error: 
 Accuracy: 40.7%, Avg loss: 0.033456 

Epoch 3
------------------------
loss: 2.132123 [    0/60000]
loss: 2.087005 [ 6400/60000]
loss: 2.045387 [12800/60000]
loss: 2.084857 [19200/60000]
loss: 2.026437 [25600/60000]
loss: 2.045294 [32000/60000]
loss: 1.997889 [38400/60000]
loss: 1.983847

## Lưu models
Một cách phổ biến để lưu một mô hình là tuần tự hóa từ điển trạng thái bên trong (chứa các tham số của mô hình).

In [9]:
torch.save(model.state_dict(), 'model.pth')
print("Save Pytorch Model State to model.pth")

Save Pytorch Model State to model.pth


## Tải models
Quá trình tải một mô hình bao gồm tạo lại cấu trúc mô hình và tải từ điển trạng thái vào đó.

In [10]:
model = NeuralNetwork()
model.load_state_dict(torch.load('model.pth'))

<All keys matched successfully>

Mô hình này hiện có thể được sử dụng để đưa ra dự đoán.

In [11]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
